In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
from config import CORPUSE_DATA_FILEPATH

In [34]:
# read it in to inspect it
with open(CORPUSE_DATA_FILEPATH, 'r', encoding='utf-8') as f:
    text = f.read()

In [35]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  394290


In [36]:
# let's look at the first 1000 characters
print(text[:1000])


 
 

 
 
 
 
 
 
 
 
 

 1. Óbito do   autor 
 A lgum tempo hesitei se devia abrir estas memórias pelo princípio ou pelo fim, isto é, se poria em primeiro lugar o meu nascimento ou a minha morte. Suposto o uso vulgar seja começar pelo nascimento, duas considerações me levaram a adotar diferente método: a primeira é que eu não sou propriamente um autor defunto, mas um defunto autor, para quem a campa foi outro berço; a segunda é que  o escrito ficaria assim mais galante e mais novo. Moisés, que também contou a sua morte, não a pôs no intróito, mas no cabo: diferença radical entre este livro e o Pentateuco. 
 Dito isto, expirei às duas horas da tarde de uma sexta-feira do mês de agosto de 1869, na minha bela chácara de Catumbi. Tinha uns sessenta e quatro anos, rijos e prósperos, era solteiro, possuía cerca de trezentos contos e fui acompanhado ao cemitério por onze  amigos. Onze amigos! Verdade é que não houve cartas nem anúncios. Acresce que chovia  –  peneirava uma chuvinha miúda, tr

In [37]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

	
 !&()*,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXZ[]_abcdefghijklmnopqrstuvwxyz°´ÀÁÂÃÇÉÍÓÚàáâãçèéêíñóôõúûü–—‘’“”…
113


In [39]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("pelo fim"))
print(decode(encode("pelo fim")))

[68, 57, 64, 67, 2, 58, 61, 65]
pelo fim


In [41]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

/home/cafe/.envs/nanoGPT/lib/python3.10/site-packages/torch/_subclasses/functional_tensor.py:258: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


torch.Size([394290]) torch.int64
tensor([  1,   2,   1,   2,   1,   1,   2,   1,   2,   1,   2,   1,   2,   1,
          2,   1,   2,   1,   2,   1,   2,   1,   2,   1,   1,   2,  13,  10,
          2,  88,  54,  61,  72,  67,   2,  56,  67,   2,   2,   2,  53,  73,
         72,  67,  70,   2,   1,   2,  25,   2,  64,  59,  73,  65,   2,  72,
         57,  65,  68,  67,   2,  60,  57,  71,  61,  72,  57,  61,   2,  71,
         57,   2,  56,  57,  74,  61,  53,   2,  53,  54,  70,  61,  70,   2,
         57,  71,  72,  53,  71,   2,  65,  57,  65, 100,  70,  61,  53,  71,
          2,  68,  57,  64,  67,   2,  68,  70,  61,  66,  55,  98,  68,  61,
         67,   2,  67,  73,   2,  68,  57,  64,  67,   2,  58,  61,  65,   8,
          2,  61,  71,  72,  67,   2,  96,   8,   2,  71,  57,   2,  68,  67,
         70,  61,  53,   2,  57,  65,   2,  68,  70,  61,  65,  57,  61,  70,
         67,   2,  64,  73,  59,  53,  70,   2,  67,   2,  65,  57,  73,   2,
         66,  53,  71,  55,  61

In [42]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [44]:
block_size = 8
train_data[:block_size+1]

tensor([1, 2, 1, 2, 1, 1, 2, 1, 2])

In [45]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([1]) the target: 2
when input is tensor([1, 2]) the target: 1
when input is tensor([1, 2, 1]) the target: 2
when input is tensor([1, 2, 1, 2]) the target: 1
when input is tensor([1, 2, 1, 2, 1]) the target: 1
when input is tensor([1, 2, 1, 2, 1, 1]) the target: 2
when input is tensor([1, 2, 1, 2, 1, 1, 2]) the target: 1
when input is tensor([1, 2, 1, 2, 1, 1, 2, 1]) the target: 2


In [46]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[67,  2, 71, 57, 70, 61, 53,  2],
        [57, 65, 68, 67, 71,  2, 56, 57],
        [97, 66, 55, 61, 53, 71, 23,  2],
        [56, 57,  2, 65, 73, 64, 60, 57]])
targets:
torch.Size([4, 8])
tensor([[ 2, 71, 57, 70, 61, 53,  2, 65],
        [65, 68, 67, 71,  2, 56, 57, 68],
        [66, 55, 61, 53, 71, 23,  2, 72],
        [57,  2, 65, 73, 64, 60, 57, 70]])
----
when input is [67] the target: 2
when input is [67, 2] the target: 71
when input is [67, 2, 71] the target: 57
when input is [67, 2, 71, 57] the target: 70
when input is [67, 2, 71, 57, 70] the target: 61
when input is [67, 2, 71, 57, 70, 61] the target: 53
when input is [67, 2, 71, 57, 70, 61, 53] the target: 2
when input is [67, 2, 71, 57, 70, 61, 53, 2] the target: 65
when input is [57] the target: 65
when input is [57, 65] the target: 68
when input is [57, 65, 68] the target: 67
when input is [57, 65, 68, 67] the target: 71
when input is [57, 65, 68, 67, 71] the target: 2
when input is [57, 